In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
# from causalml.inference.meta import BaseDRLearner
# from causalml.inference.tree import CausalForest
# from causalml.inference.bart import BaseCausalModel
# from causalml.propensity import compute_propensity_score

from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

def estimate_causal_forest_econml(df, outcome, treatment, covariates):
    Y = df[outcome].values
    T = df[treatment].values
    X = df[covariates].values

    model = CausalForestDML(
        model_y=RandomForestRegressor(),
        model_t=LogisticRegression(),
        discrete_treatment=True,
        random_state=42
    )
    model.fit(Y, T, X=X)
    ate_cf = model.ate(X)
    return ate_cf


In [14]:
import os
path = '/media/ak/DataOnly1/NIPSProjectData'
files = [f for f in os.listdir(path) if '.csv' in f]
filePath = os.path.join(path, files[1])
df =pd.read_csv(filePath)

In [21]:

def create_osrct_dataset(
    df, outcome, treatment, covariates, biasing_covariates,
    alpha=2.0, seed=42, verbose=True, bias_model_type="linear"
):
    rng = np.random.default_rng(seed)

    # Validate binary treatment
    if not set(df[treatment].unique()).issubset({0, 1}):
        raise ValueError("Treatment must be binary (0 or 1).")

    for col in biasing_covariates:
        if df[col].isnull().any():
            raise ValueError(f"Missing values in {col}")
        if df[col].nunique() <= 1:
            raise ValueError(f"Low variance in {col}")

    # Train biasing model
    if bias_model_type == "linear":
        model = LogisticRegression()
    elif bias_model_type == "nonlinear":
        model = GradientBoostingClassifier(n_estimators=100, max_depth=3)
    else:
        raise ValueError("bias_model_type must be 'linear' or 'nonlinear'")

    model.fit(df[biasing_covariates], df[treatment])
    probs_raw = model.predict_proba(df[biasing_covariates])[:, 1]
    probs_raw = np.clip(probs_raw, 1e-10, 1 - 1e-10)
    logits = np.log(probs_raw / (1 - probs_raw))
    probs = 1 / (1 + np.exp(-alpha * logits))
    sampled_treatment = rng.binomial(1, probs)

    # Select rows where sampled treatment == actual treatment
    mask = sampled_treatment == df[treatment]
    osrct_df = df[mask].copy()
    complement_df = df[~mask].copy()

    true_ate = df[df[treatment] == 1][outcome].mean() - df[df[treatment] == 0][outcome].mean()

    if verbose:
        print(f"\n📊 True ATE (RCT): {true_ate:.4f}")
        print(f"OSRCT sample size: {len(osrct_df)} ({len(osrct_df)/len(df):.1%})")
        print(f"Complement sample size: {len(complement_df)} ({len(complement_df)/len(df):.1%})")
        print(f"Treatment rate OSRCT: {osrct_df[treatment].mean():.2f}")
        print(f"Treatment rate Complement: {complement_df[treatment].mean():.2f}")

    return osrct_df, complement_df, true_ate

def estimate_ate_econml(df, outcome, treatment, covariates, methods=["cf", "dr", "linear_dml", "ipw"]):
    X = df[covariates].values
    T = df[treatment].values
    Y = df[outcome].values

    results = {}

    # Causal Forest DML
    if "cf" in methods:
        model_cf = CausalForestDML(
            model_y=RandomForestRegressor(),
            model_t=LogisticRegression(),
            discrete_treatment=True,
            random_state=42
        )
        model_cf.fit(Y, T, X=X)
        results["Causal Forest"] = model_cf.ate(X)

    # Linear DML (doubly robust linear model)
    if "linear_dml" in methods:
        model_linear = LinearDML(
            model_y=RandomForestRegressor(),
            model_t=LogisticRegression(),
            discrete_treatment=True,
            random_state=42
        )
        model_linear.fit(Y, T, X=X)
        results["Linear DML"] = model_linear.ate(X)

    # DR Learner
    if "dr" in methods:
        model_dr = DRLearner(
            model_regression=RandomForestRegressor(),
            model_propensity=LogisticRegression()
        )
        model_dr.fit(Y, T, X=X)
        results["DR Learner"] = model_dr.ate(X)

    # Inverse Propensity Weighting (manual)
    if "ipw" in methods:
        prop_model = LogisticRegression()
        prop_model.fit(X, T)
        ps = prop_model.predict_proba(X)[:, 1]
        ps = np.clip(ps, 1e-3, 1 - 1e-3)
        weights = T / ps + (1 - T) / (1 - ps)
        results["IPW"] = np.mean(weights * (T * Y / ps - (1 - T) * Y / (1 - ps)))

    return results
# def estimate_ate_methods(df, outcome, treatment, covariates, methods=["ipw", "dr", "bart", "cf"]):
#     print("\n🧪 Estimating ATE with causalml models...\n")
#     results = {}
#     X, T, Y = df[covariates], df[treatment].values, df[outcome].values

#     from sklearn.linear_model import LogisticRegression

#     if "ipw" in methods:
#         prop_model = LogisticRegression()
#         prop_model.fit(X, T)
#         ps = prop_model.predict_proba(X)[:, 1]
#         ps = np.clip(ps, 1e-3, 1 - 1e-3)
#         weights = T / ps + (1 - T) / (1 - ps)
#         results["IPW"] = np.mean(weights * (T * Y / ps - (1 - T) * Y / (1 - ps)))


#     if "dr" in methods:
#         learner = BaseDRLearner()
#         results["DR"] = learner.estimate_ate(X=X, treatment=T, y=Y)["ate"]

#     if "bart" in methods:
#         model = BaseCausalModel(learner="bart")
#         model.fit(X=X, treatment=T, y=Y)
#         results["BART"] = model.estimate_ate(X)["ate"]

#     if "cf" in methods:
#         model = CausalForest()
#         model.fit(X=X, treatment=T, y=Y)
#         results["Causal Forest"] = model.estimate_ate(X)["ate"]

#     return results


def visualize_osrct_distributions(df, osrct_df, complement_df, outcome, treatment, biasing_covariates):
    sns.set(style="whitegrid")

    # 1. Treatment Proportions
    rates = pd.DataFrame({
        "Original": df[treatment].value_counts(normalize=True),
        "OSRCT": osrct_df[treatment].value_counts(normalize=True),
        "Complement": complement_df[treatment].value_counts(normalize=True)
    }).T
    rates.plot(kind="bar", stacked=True)
    plt.title("📊 Treatment Distribution (Original vs OSRCT vs Complement)")
    plt.ylabel("Proportion")
    plt.tight_layout()
    plt.show()

    # 2. KDE plots for each biasing covariate
    for cov in biasing_covariates:
        plt.figure(figsize=(6, 4))
        sns.kdeplot(df[cov], label="Original", lw=2)
        sns.kdeplot(osrct_df[cov], label="OSRCT", lw=2, linestyle="--")
        sns.kdeplot(complement_df[cov], label="Complement", lw=2, linestyle=":")
        plt.title(f"📈 Distribution of {cov}")
        plt.legend()
        plt.tight_layout()
        plt.show()

    # 3. Outcome histograms by treatment
    plt.figure(figsize=(8, 4))
    sns.histplot(data=osrct_df, x=outcome, hue=treatment, element="step", bins=30, kde=True)
    plt.title("Outcome Distribution by Treatment (OSRCT)")
    plt.tight_layout()
    plt.show()



In [ ]:

# === Main Driver ===
if __name__ == "__main__":
    # FILE_PATH = "diabetes_data.csv"
    # df = pd.read_csv(FILE_PATH)

    outcome = "Diabetes_012"
    treatment = "HighBP"
    biasing_covariates = ["Age", "BMI", "Income"]
    covariates = [c for c in df.columns if c not in [outcome, treatment]]

    osrct_df, complement_df, true_ate = create_osrct_dataset(
        df, outcome, treatment, covariates, biasing_covariates,
        alpha=3.0, bias_model_type="nonlinear", verbose=True
    )

    # results = estimate_ate_methods(osrct_df, outcome, treatment, covariates)
    # comp_ipw = estimate_ate_methods(complement_df, outcome, treatment, covariates, methods=["ipw"])
    results = estimate_ate_econml(
    osrct_df,
    outcome="Diabetes_012",
    treatment="HighBP",
    covariates=covariates,
    methods=["ipw", "cf", "dr", "linear_dml"]
)

    print("\n📊 Estimated ATEs on OSRCT Sample:")
    for method, val in results.items():
        print(f"  {method}: {val:.4f}")

    print("\n📄 Complement Sample ATE (IPW only):")
    print(f"  IPW: {comp_ipw['IPW']:.4f}")

    print(f"\n🎯 True ATE from RCT: {true_ate:.4f}")

    # Show full visual analysis
    visualize_osrct_distributions(df, osrct_df, complement_df, outcome, treatment, biasing_covariates)


In [ ]:

# === Main Driver ===
if __name__ == "__main__":
    # FILE_PATH = "diabetes_data.csv"
    # df = pd.read_csv(FILE_PATH)

    outcome = "Diabetes_012"
    treatment = "HighBP"
    biasing_covariates = ["Age", "BMI", "Income"]
    covariates = [c for c in df.columns if c not in [outcome, treatment]]

    osrct_df, complement_df, true_ate = create_osrct_dataset(
        df, outcome, treatment, covariates, biasing_covariates,
        alpha=3.0, bias_model_type="nonlinear", verbose=True
    )

    # results = estimate_ate_methods(osrct_df, outcome, treatment, covariates)
    # comp_ipw = estimate_ate_methods(complement_df, outcome, treatment, covariates, methods=["ipw"])

    # print("\n📊 Estimated ATEs on OSRCT Sample:")
    # for method, val in results.items():
    #     print(f"  {method}: {val:.4f}")

    # print("\n📄 Complement Sample ATE (IPW only):")
    # print(f"  IPW: {comp_ipw['IPW']:.4f}")

    # print(f"\n🎯 True ATE from RCT: {true_ate:.4f}")

    # # Show full visual analysis
    # visualize_osrct_distributions(df, osrct_df, complement_df, outcome, treatment, biasing_covariates)


In [ ]:

    # === 2. Define Variables ===
    outcome = "Diabetes_012"
    treatment = "HighBP"
    biasing_covariates = ["Age", "BMI", "Income"]
    covariates = [c for c in df.columns if c not in [outcome, treatment]]

    # === 3. Generate OSRCT + Complement ===
    osrct_df, complement_df, true_ate = create_osrct_dataset(
        df=df,
        outcome=outcome,
        treatment=treatment,
        covariates=covariates,
        biasing_covariates=biasing_covariates,
        alpha=3.0,
        bias_model_type="nonlinear",
        verbose=True
    )

    # === 4. Estimate ATEs (EconML methods only) ===
    results = estimate_ate_econml(
        df=osrct_df,
        outcome=outcome,
        treatment=treatment,
        covariates=covariates,
        methods=["ipw", "cf", "dr", "linear_dml"]
    )

    comp_results = estimate_ate_econml(
        df=complement_df,
        outcome=outcome,
        treatment=treatment,
        covariates=covariates,
        methods=["ipw"]
    )

    # === 5. Display Results ===
    print("\n📊 Estimated ATEs on OSRCT Sample:")
    for method, val in results.items():
        print(f"  {method}: {val:.4f}")

    print("\n📄 Complement Sample ATE (IPW only):")
    print(f"  IPW: {comp_results['IPW']:.4f}")

    print(f"\n🎯 True ATE from RCT: {true_ate:.4f}")

    # === 6. Visualizations ===
    visualize_osrct_distributions(
        df, osrct_df, complement_df,
        outcome, treatment, biasing_covariates
    )